<a href="https://colab.research.google.com/github/Tegarbangkit21/glcm-lung-cancer-detector/blob/main/PRAK_PC_Tegar009_Kaka029_Dappa053_kanker_Paru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link dataset : [link dataset kanker paru](https://drive.google.com/drive/folders/16bg01R7LdmhfjU1kC2TteoHEXxtybFHl?usp=sharing)

In [ ]:
!pip install gradio opencv-python-headless

In [ ]:
from google.colab import files
files.upload()  # Upload file kaggle.json dulu
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving lung_colon_image_set.zip to lung_colon_image_set.zip
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
import zipfile
import os

zip_path = '/content/lung_colon_image_set.zip'
extract_path = '/content/lung_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Cek isi folder
os.listdir(extract_path)

['lung_colon_image_set']

In [ ]:
import shutil
import random
import os

# Pastikan path ini sesuai hasil ekstraksi
src_kanker = '/content/lung_data/lung_colon_image_set/lung_image_sets/lung_aca'
src_normal = '/content/lung_data/lung_colon_image_set/lung_image_sets/lung_n'

# Buat folder sample target
sample_dir = '/content/sample_data_35'
os.makedirs(os.path.join(sample_dir, 'kanker'), exist_ok=True)
os.makedirs(os.path.join(sample_dir, 'normal'), exist_ok=True)

# Copy sample gambar
for img in random.sample(os.listdir(src_kanker), 18):
    shutil.copy(os.path.join(src_kanker, img), os.path.join(sample_dir, 'kanker'))

for img in random.sample(os.listdir(src_normal), 17):
    shutil.copy(os.path.join(src_normal, img), os.path.join(sample_dir, 'normal'))

print("Jumlah gambar kanker:", len(os.listdir(os.path.join(sample_dir, 'kanker'))))
print("Jumlah gambar normal:", len(os.listdir(os.path.join(sample_dir, 'normal'))))

Jumlah gambar kanker: 18
Jumlah gambar normal: 17


In [ ]:
# Install library
!pip install gradio opencv-python-headless mahotas scikit-learn --quiet

import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mahotas
from skimage.feature import graycomatrix, graycoprops
import io
import PIL.Image
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# --- Fungsi GLCM ---
def extract_glcm_features(gray):
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    return {
        'Contrast': round(contrast, 4),
        'Correlation': round(correlation, 4),
        'Energy': round(energy, 4),
        'Homogeneity': round(homogeneity, 4)
    }

# --- Ekstraksi fitur dari dataset sample ---
def load_dataset_and_train(sample_path):
    data = []
    labels = []

    for label in ['normal', 'kanker']:
        folder = os.path.join(sample_path, label)
        for filename in os.listdir(folder):
            path = os.path.join(folder, filename)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                feats = extract_glcm_features(img)
                data.append(feats)
                labels.append(label)

    df = pd.DataFrame(data)
    df['Label'] = labels

    X = df.drop('Label', axis=1)
    y = df['Label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    print("Hasil evaluasi model:")
    print(classification_report(y_test, model.predict(X_test)))

    return model

# Path ke data latih sample
sample_data_path = '/content/sample_data_35'
model = load_dataset_and_train(sample_data_path)

# --- Fungsi prediksi + visualisasi ---
def process_and_classify(input_img):
    image = np.array(input_img)

    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image

    # Histogram
    hist_fig, ax = plt.subplots()
    ax.hist(gray.ravel(), bins=256, color='gray')
    ax.set_title("Histogram")
    ax.set_xlabel("Intensitas")
    ax.set_ylabel("Jumlah")
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    hist_img = PIL.Image.open(buf)

    # Filter median
    median = cv2.medianBlur(gray, 5)

    # Segmentasi Canny
    edges = cv2.Canny(median, 100, 200)

    # Ekstraksi fitur dan prediksi
    glcm_feats = extract_glcm_features(gray)
    X = pd.DataFrame([glcm_feats])
    prediction = model.predict(X)[0]

    return gray, median, edges, hist_img, glcm_feats, prediction

# --- Interface Gradio ---
iface = gr.Interface(
    fn=process_and_classify,
    inputs=gr.Image(type="pil", label="Upload Gambar"),
    outputs=[
        gr.Image(label="Grayscale"),
        gr.Image(label="Median Filter"),
        gr.Image(label="Canny Edge"),
        gr.Image(label="Histogram"),
        gr.JSON(label="GLCM Features"),
        gr.Text(label="Prediksi: Kanker / Normal")
    ],
    title = "Analisis Tekstur Citra Paru dengan GLCM: Grayscale, Filter Median, Canny, Histogram, dan Ekstraksi Fitur",
    description="Model dilatih dari data sample. Upload gambar mikroskopik untuk analisis tekstur dan prediksi klasifikasi."
)

iface.launch()


Hasil evaluasi model:
              precision    recall  f1-score   support

      kanker       0.50      0.75      0.60         4
      normal       0.00      0.00      0.00         3

    accuracy                           0.43         7
   macro avg       0.25      0.38      0.30         7
weighted avg       0.29      0.43      0.34         7

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb2c2292b79d9bbb91.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
